# Initial Cleaning/ setup procedures


#### _1. Import libraries and set path to pkl file_

In [2]:
import pandas as pd
import os
from datetime import date, timedelta
import datetime


ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
pkl_loc = os.path.join(ROOT_DIR, 'data', 'data','weathersmallraw.pkl')
pkl_loc

'C:\\Users\\paull\\OneDrive - Scandinavian Tobacco Group A S\\Documents\\GitHub\\Weatherman\\data\\data\\weathersmallraw.pkl'

### _2. Create dataframe and run basic eda_

In [3]:
df = pd.read_pickle(pkl_loc)
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3765246 entries, ('0CNUO', Timestamp('2022-01-01 00:00:00')) to ('ZYITU', Timestamp('2022-09-22 23:00:00'))
Data columns (total 11 columns):
 #   Column  Dtype  
---  ------  -----  
 0   temp    float64
 1   dwpt    float64
 2   rhum    float64
 3   prcp    float64
 4   snow    float64
 5   wdir    float64
 6   wspd    float64
 7   wpgt    float64
 8   pres    float64
 9   tsun    float64
 10  coco    float64
dtypes: float64(11)
memory usage: 330.7+ MB


In [4]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.describe()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,3765202.000,3765187.000,3765207.000,3744744.000,0.000,3765167.000,3765177.000,0.000,3765066.000,0.000,1614583.000
mean,9.246,3.645,71.106,0.116,NaN,184.294,13.762,NaN,1015.720,NaN,3.003
std,13.836,13.119,18.702,1.052,NaN,111.576,9.344,NaN,7.956,NaN,2.788
min,-43.000,-52.900,0.000,0.000,NaN,0.000,0.000,NaN,970.300,NaN,1.000
25%,-0.700,-5.700,58.000,0.000,NaN,91.000,7.600,NaN,1011.000,NaN,1.000
50%,12.200,6.200,73.000,0.000,NaN,191.000,13.000,NaN,1015.800,NaN,3.000
75%,20.300,14.800,86.000,0.000,NaN,280.000,18.400,NaN,1020.200,NaN,3.000
max,60.000,60.000,100.000,332.000,NaN,360.000,189.000,NaN,1058.700,NaN,25.000


In [5]:
df.shape

(3765246, 11)

### _3. Remove unneeded columns_


In [6]:
df.drop(columns=['snow', 'wpgt','tsun','coco'],inplace=True)
df.head()

temp    dwpt   rhum  prcp    wdir   wspd  \
station time                                                              
0CNUO   2022-01-01 00:00:00 -25.100 -28.100 76.000 0.000 271.000 12.600   
        2022-01-01 01:00:00 -25.700 -28.700 76.000 0.000 275.000 13.000   
        2022-01-01 02:00:00 -26.200 -29.300 75.000 0.000 273.000 12.600   
        2022-01-01 03:00:00 -26.100 -29.100 76.000 0.000 277.000 13.700   
        2022-01-01 04:00:00 -26.500 -29.600 75.000 0.000 275.000 13.300   

                                pres  
station time                          
0CNUO   2022-01-01 00:00:00 1020.800  
        2022-01-01 01:00:00 1021.500  
        2022-01-01 02:00:00 1022.500  
        2022-01-01 03:00:00 1022.600  
        2022-01-01 04:00:00 1023.200

In [7]:
df=df.reset_index()

### _4. Counting Stations_

In [8]:
df.groupby("station").count() #clearly some stations are missing

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres
station,,,,,,,,
0CNUO,6352,6352,6352,6352,6350,6352,6352,6352
0CO7B,6352,6352,6352,6352,6350,6352,6352,6352
0FV1F,6352,6352,6352,6352,6350,6352,6352,6352
1J1PJ,6360,6360,6360,6360,6360,6360,6360,6360
1JWST,6360,6360,6360,6360,6360,6359,6360,6360
...,...,...,...,...,...,...,...,...
ZFZUV,6360,6360,6360,6360,6358,6360,6360,6360
ZNWZW,3658,3658,3658,3658,3638,3658,3658,3658
ZUQJS,3342,3342,3342,3342,3329,3342,3342,3331


In [60]:
dfstations=df.groupby("station").count()
df_stationskeep = dfstations[dfstations['time']>5000].reset_index() #creating this to know which stations to keep later

### _5. Difference in Time_

In [10]:
dftime=df.set_index('station')

In [11]:
dftime['timediff']=dftime['time'].diff(periods=1) #difference for hours

In [12]:
dftime['timediff']=dftime['timediff']/pd.Timedelta(hours=1) #converted to hour int

In [13]:

#This is fantiastic, it looks like we are missing only data 253 times, and none of it more than 7 hours.  Seems like an idea situation.
dftime[dftime['timediff']>1]

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,timediff
station,,,,,,,,,
0CNUO,2022-05-26 12:00:00,7.200,5.800,91.000,NaN,45.000,8.600,1012.500,6.000
0CNUO,2022-05-26 16:00:00,14.100,4.400,52.000,NaN,78.000,10.400,1013.100,4.000
0CO7B,2022-05-26 12:00:00,19.200,17.000,87.000,NaN,172.000,26.600,1011.500,6.000
0CO7B,2022-05-26 16:00:00,22.200,18.200,78.000,NaN,202.000,26.600,1013.400,4.000
0FV1F,2022-05-26 12:00:00,16.700,13.400,81.000,NaN,168.000,14.800,1015.000,6.000
...,...,...,...,...,...,...,...,...,...
ZUQJS,2022-05-06 19:00:00,12.800,2.600,50.000,NaN,20.000,13.000,NaN,2.000
ZUQJS,2022-05-06 21:00:00,15.000,0.800,38.000,NaN,60.000,16.600,NaN,2.000
ZUQJS,2022-05-07 01:00:00,10.200,-0.600,47.000,NaN,70.000,7.600,NaN,4.000


In [14]:
#Looking at the data above in more detail for a specific location.
test=dftime.loc['0CNUO']

In [15]:

test[(test['time']>='2022-05-25 01:00:00') & (test['time']<'2022-05-27 01:00:00')] 
#we seem to have a couple issues, first there are missing values here, and one missing prcp goes to naan

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,timediff
station,,,,,,,,,
0CNUO,2022-05-25 01:00:00,15.100,7.600,61.000,0.000,217.000,8.600,1019.500,1.000
0CNUO,2022-05-25 02:00:00,12.500,6.800,68.000,0.000,217.000,10.400,1020.100,1.000
0CNUO,2022-05-25 03:00:00,10.900,6.200,73.000,0.000,223.000,9.400,1020.600,1.000
0CNUO,2022-05-25 04:00:00,9.900,6.100,77.000,0.000,223.000,9.400,1020.900,1.000
0CNUO,2022-05-25 05:00:00,9.300,5.500,77.000,0.000,224.000,9.400,1021.200,1.000
0CNUO,2022-05-25 06:00:00,8.700,5.100,78.000,0.000,218.000,9.000,1021.600,1.000
0CNUO,2022-05-25 07:00:00,7.900,5.000,82.000,0.000,222.000,6.800,1021.600,1.000
0CNUO,2022-05-25 08:00:00,7.200,4.800,85.000,0.000,217.000,7.600,1021.600,1.000
0CNUO,2022-05-25 09:00:00,6.800,4.800,87.000,0.000,206.000,6.500,1021.500,1.000


### _6. Null and Precep Count_

In [16]:
dftime.isnull().sum()

#lot of null for prcp


time            0
temp           44
dwpt           59
rhum           39
prcp        20502
wdir           79
wspd           69
pres          180
timediff        1
dtype: int64

In [17]:
dftime[dftime["prcp"]>0]

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,timediff
station,,,,,,,,,
0CNUO,2022-01-03 07:00:00,-20.600,-24.000,74.000,0.100,177.000,12.200,1018.400,1.000
0CNUO,2022-01-03 08:00:00,-19.900,-23.200,75.000,0.100,183.000,10.100,1018.100,1.000
0CNUO,2022-01-04 12:00:00,-19.600,-22.000,81.000,0.100,56.000,12.200,1020.200,1.000
0CNUO,2022-01-04 13:00:00,-19.400,-22.100,79.000,0.100,64.000,13.300,1019.500,1.000
0CNUO,2022-01-04 16:00:00,-18.600,-21.600,77.000,0.100,66.000,16.200,1018.400,1.000
...,...,...,...,...,...,...,...,...,...
ZYITU,2022-09-21 04:00:00,17.000,16.800,99.000,0.100,170.000,11.000,1013.000,1.000
ZYITU,2022-09-21 05:00:00,17.400,17.200,99.000,0.100,0.000,0.000,1012.100,1.000
ZYITU,2022-09-21 06:00:00,17.000,16.800,99.000,0.800,150.000,6.000,1012.000,1.000


In [18]:
dftest=dftime.loc['KARB0']

In [19]:
dftest[dftest["prcp"]>0]

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,timediff
station,,,,,,,,,
KARB0,2022-01-01 22:00:00,-0.600,-2.200,89.000,0.500,50.000,16.600,1008.500,1.000
KARB0,2022-01-01 23:00:00,-1.100,-2.800,88.000,0.200,30.000,14.800,1009.000,1.000
KARB0,2022-01-02 00:00:00,-2.200,-3.900,88.000,0.600,50.000,16.600,1010.000,1.000
KARB0,2022-01-02 01:00:00,-2.800,-5.600,81.000,0.700,30.000,16.600,1010.200,1.000
KARB0,2022-01-02 02:00:00,-3.300,-5.600,84.000,0.400,30.000,24.100,1009.900,1.000
...,...,...,...,...,...,...,...,...,...
KARB0,2022-09-17 21:00:00,27.800,15.500,47.000,0.100,250.000,16.600,1016.800,1.000
KARB0,2022-09-19 11:00:00,17.200,16.100,93.000,3.000,280.000,7.600,1012.100,1.000
KARB0,2022-09-21 06:00:00,17.000,15.300,90.000,1.100,200.000,11.000,1013.000,1.000


In [20]:
dftime

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,timediff
station,,,,,,,,,
0CNUO,2022-01-01 00:00:00,-25.100,-28.100,76.000,0.000,271.000,12.600,1020.800,NaN
0CNUO,2022-01-01 01:00:00,-25.700,-28.700,76.000,0.000,275.000,13.000,1021.500,1.000
0CNUO,2022-01-01 02:00:00,-26.200,-29.300,75.000,0.000,273.000,12.600,1022.500,1.000
0CNUO,2022-01-01 03:00:00,-26.100,-29.100,76.000,0.000,277.000,13.700,1022.600,1.000
0CNUO,2022-01-01 04:00:00,-26.500,-29.600,75.000,0.000,275.000,13.300,1023.200,1.000
...,...,...,...,...,...,...,...,...,...
ZYITU,2022-09-22 19:00:00,16.400,4.100,44.000,0.000,329.000,32.000,1015.800,1.000
ZYITU,2022-09-22 20:00:00,15.900,3.700,44.000,0.000,326.000,30.600,1016.100,1.000
ZYITU,2022-09-22 21:00:00,16.000,3.400,43.000,0.000,331.000,29.200,1016.400,1.000


In [21]:
#cln_pkl_loc = os.path.join(ROOT_DIR, 'data_cleaning','cleanweathersmall.pkl')
#df.to_pickle(cln_pkl_loc)


### _6. Merging in new time_

In [37]:
time_range = pd.date_range('2022-01-01 00:00:00', '2022-09-22 23:00:00', freq='H')

In [74]:
df_time=pd.DataFrame(time_range)

In [75]:
df_time.columns =['time']

In [76]:
df_station = pd.DataFrame(df.station.unique())
df_station.columns=['station']


In [77]:
#merging to limit to the 575 stations which have data back to Jan.
df_station=df_station.merge(df_stationskeep.station, left_on='station',right_on='station') 

In [78]:
df_time_station=df_station.merge(df_time, how='cross')

In [79]:
df_time_station

,station,time
0,0CNUO,2022-01-01 00:00:00
1,0CNUO,2022-01-01 01:00:00
2,0CNUO,2022-01-01 02:00:00
3,0CNUO,2022-01-01 03:00:00
4,0CNUO,2022-01-01 04:00:00
...,...,...
3656995,ZWC6W,2022-09-22 19:00:00
3656996,ZWC6W,2022-09-22 20:00:00
3656997,ZWC6W,2022-09-22 21:00:00
3656998,ZWC6W,2022-09-22 22:00:00


In [84]:
df=df_time_station.merge(df,left_on=['station', 'time'], right_on=['station','time'], how='left') #new df with nulls 

In [85]:
df

,station,time,temp,dwpt,rhum,prcp,wdir,wspd,pres
0,0CNUO,2022-01-01 00:00:00,-25.100,-28.100,76.000,0.000,271.000,12.600,1020.800
1,0CNUO,2022-01-01 01:00:00,-25.700,-28.700,76.000,0.000,275.000,13.000,1021.500
2,0CNUO,2022-01-01 02:00:00,-26.200,-29.300,75.000,0.000,273.000,12.600,1022.500
3,0CNUO,2022-01-01 03:00:00,-26.100,-29.100,76.000,0.000,277.000,13.700,1022.600
4,0CNUO,2022-01-01 04:00:00,-26.500,-29.600,75.000,0.000,275.000,13.300,1023.200
...,...,...,...,...,...,...,...,...,...
3656995,ZWC6W,2022-09-22 19:00:00,8.900,-2.700,44.000,0.000,324.000,22.300,1017.300
3656996,ZWC6W,2022-09-22 20:00:00,10.300,-3.800,37.000,0.000,328.000,22.700,1017.000
3656997,ZWC6W,2022-09-22 21:00:00,9.200,-1.500,47.000,0.000,322.000,23.800,1017.700
3656998,ZWC6W,2022-09-22 22:00:00,8.900,-1.500,48.000,0.000,326.000,21.200,1018.000
